### Učitavanje biblioteka

In [65]:
import numpy as np
import pandas as pd 
import surprise as sp
from sklearn.preprocessing import LabelEncoder

### Učitavanje skupa podataka i pretprocesiranje

In [66]:
# Učitavanje skupa podataka

def loaddata(filename):
    df = pd.read_csv(f'{filename}.csv',sep=',',error_bad_lines=False, warn_bad_lines=False, encoding='latin-1')
    return df

amazon_data = loaddata('skup_amazon/fashionMeta')
data = loaddata('skup_amazon/fashion')

/tmp/ipykernel_22468/837364138.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(f'{filename}.csv',sep=',',error_bad_lines=False, warn_bad_lines=False, encoding='latin-1')
/tmp/ipykernel_22468/837364138.py:4: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(f'{filename}.csv',sep=',',error_bad_lines=False, warn_bad_lines=False, encoding='latin-1')
/tmp/ipykernel_22468/837364138.py:4: DtypeWarning: Columns (149,150,151,152,153,154,155,156,157,158,159,160,161,230,231,232,242,243,244,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,3

In [67]:
amazon_data.head(2)

,title,brand,feature/0,feature/1,rank,date,asin,imageURL/0,imageURLHighRes/0,imageURL/1,...,similar_item/4/features/Item Weight,similar_item/4/features/Size,similar_item/4/features/character,similar_item/5/features/Color,similar_item/5/features/Item Dimensions,similar_item/5/features/Item Weight,similar_item/5/features/Size,similar_item/5/features/character,tech1/Folding,tech1/Item Dimensions L x W x H
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),Product Dimensions:\n \n8.7...,Shipping Weight:\n \n2.4 po...,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Izdvajanje atributa
filtered_amazon_data = amazon_data[['asin','title', 'brand', 'feature/0', 'feature/1']]

# Uklanjanje NaN vrednosti za sve atribute
cleaned_amazon_data = filtered_amazon_data.rename(columns={'feature/0': 'dimensions', 'feature/1' : 'weight'}).dropna()

# Uklanjanje duplikata
cleaned_amazon_data.drop_duplicates(inplace=True)
cleaned_amazon_data

cleaned_amazon_data.head(2)

,asin,title,brand,dimensions,weight
0,0764443682,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),Product Dimensions:\n \n8.7...,Shipping Weight:\n \n2.4 po...
2,1940280001,Magical Things I Really Do Do Too!,Christopher Manos,Package Dimensions:\n \n8.5...,Shipping Weight:\n \n6.1 ou...


In [69]:
encoded_amazon_data = cleaned_amazon_data.copy()

item_encoder = LabelEncoder()
item_encoder.fit(encoded_amazon_data['asin'].values)
n_items = len(item_encoder.classes_)

encoded_amazon_data["itemID"] = item_encoder.transform(encoded_amazon_data['asin'])

encoded_amazon_data.head(2)

,asin,title,brand,dimensions,weight,itemID
0,0764443682,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),Product Dimensions:\n \n8.7...,Shipping Weight:\n \n2.4 po...,0
2,1940280001,Magical Things I Really Do Do Too!,Christopher Manos,Package Dimensions:\n \n8.5...,Shipping Weight:\n \n6.1 ou...,1


In [70]:
items_all = encoded_amazon_data.assign(id=pd.Series(range(1, encoded_amazon_data.shape[0]+1), dtype=int, index=encoded_amazon_data.index))
items_all.head(2)

,asin,title,brand,dimensions,weight,itemID,id
0,0764443682,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),Product Dimensions:\n \n8.7...,Shipping Weight:\n \n2.4 po...,0,1
2,1940280001,Magical Things I Really Do Do Too!,Christopher Manos,Package Dimensions:\n \n8.5...,Shipping Weight:\n \n6.1 ou...,1,2


In [71]:
data.head(2)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,image/6,image/7,image/8,image/9,image/10,image/11,image/12,image/13,style/Size Name:,style/Style:
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Izdvajanje atributa
filtered_data = data[['reviewerID', 'reviewerName', 'asin', 'reviewText', 'summary', 'overall']]

# Uklanjanje NaN vrednosti za sve atribute
cleaned_data = filtered_data.dropna()

# Uklanjanje duplikata
cleaned_data.drop_duplicates(inplace=True)
cleaned_data

cleaned_data.head(2)

/tmp/ipykernel_22468/4291965330.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data.drop_duplicates(inplace=True)


,reviewerID,reviewerName,asin,reviewText,summary,overall
0,A1D4G1SNUZWQOT,Tracy,7106116521,Exactly what I needed.,perfect replacements!!,5
1,A3DDWDH9PX2YX2,Sonja Lau,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",2


In [73]:
encoded_data = cleaned_data.copy()

user_encoder = LabelEncoder()
user_encoder.fit(encoded_data['reviewerID'].values)
n_users = len(user_encoder.classes_)

encoded_data["userID"] = user_encoder.transform(encoded_data['reviewerID'])
encoded_data.head(2)

,reviewerID,reviewerName,asin,reviewText,summary,overall,userID
0,A1D4G1SNUZWQOT,Tracy,7106116521,Exactly what I needed.,perfect replacements!!,5,1840
1,A3DDWDH9PX2YX2,Sonja Lau,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",2,11443


In [74]:
users_all = encoded_data.assign(id=pd.Series(range(1, encoded_data.shape[0]+1), dtype=int, index=encoded_data.index))
users_all.head(2)

,reviewerID,reviewerName,asin,reviewText,summary,overall,userID,id
0,A1D4G1SNUZWQOT,Tracy,7106116521,Exactly what I needed.,perfect replacements!!,5,1840,1
1,A3DDWDH9PX2YX2,Sonja Lau,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",2,11443,2


In [75]:
# Koliko interakcija je vezano za svakog korisnika

cleaned_data['reviewerID'].value_counts()

A3W11493KS6Z2L    5
A3IDT5LC91YJJU    4
A1UF8LF658016W    4
A17EOTC7REOQKA    3
A33ZQAI5SUB533    3
                 ..
A1G5RTYYX13TTC    1
A2JO75H05ZRHYJ    1
A29BVA1UHTER23    1
A2QVXKMAKV7GUL    1
A2Y39N7FDHEPNY    1
Name: reviewerID, Length: 18292, dtype: int64

In [76]:
# Spajanje tabela

ratings_all = users_all.merge(items_all, on="asin")[['reviewerID', 'userID', 'asin', 'itemID', 'overall']]
ratings_all.head(2)

,reviewerID,userID,asin,itemID,overall
0,A1BB77SEBQT8VX,1591,B00007GDFV,11,3
1,AHWOW7D1ABO9C,15861,B00007GDFV,11,3


In [77]:
# Koliko ocena je zabelezeno za svaku od stavki

ratings_all['itemID'].value_counts()

160    715
90     319
193    235
314    226
333    185
      ... 
70       1
75       1
89       1
106      1
91       1
Name: itemID, Length: 62, dtype: int64

In [78]:
# Dodavanje novog reda u ratings_all dataframe, kako bi kasnije bio iskorišćen model zasnovan na sadržaju prilikom
# poziva metoda predict_switching, koji bira algoritam na osnovu broja postojećih ocena za dati predmet (stavku).
# Nakon izvršenja koda ispod, u ratings_all trebaju biti dva reda sa istim korisnikom/stavkom.

new_row = {'userID':3958, 'reviewerID':'A1TEZWIOUZP5X5', 'itemID':50, 'asin':'B0009MZWKW', 'overall':2}
#ratings_all = ratings_all.append(new_row, ignore_index=True)

new_row_2 = {'userID':2000, 'reviewerID':'A1TEZWIOUZP5X5', 'itemID':106, 'asin':'B0009MZWKW', 'overall':2}

new_df = pd.DataFrame([new_row])
ratings_all = pd.concat([ratings_all, new_df], axis=0, ignore_index=True)

new_df_2 = pd.DataFrame([new_row_2])
ratings_all = pd.concat([ratings_all, new_df_2], axis=0, ignore_index=True)

ratings_all[ratings_all['userID'] == 3958]

#ratings[ratings['itemID'] == item_id]
#ratings_all[ratings_all['userID'] == 13710]

,reviewerID,userID,asin,itemID,overall
932,A1TEZWIOUZP5X5,3958,B0009MZWKW,106,5
3927,A1TEZWIOUZP5X5,3958,B0009MZWKW,50,2


In [79]:
ratings_all[ratings_all['itemID'] == 106]

,reviewerID,userID,asin,itemID,overall
932,A1TEZWIOUZP5X5,3958,B0009MZWKW,106,5
3928,A1TEZWIOUZP5X5,2000,B0009MZWKW,106,2


### Implementacija različitih pristupa

In [80]:
##### CF korak po korak #####

In [81]:
# ratings_all[ratings_all['itemID'] == 106]

In [82]:
# users_all = users_all[['userID', 'reviewerID', 'reviewText', 'summary']]
# users_all

In [83]:
# ratings_all[~is_target][['userID', 'itemID', 'overall']]

In [84]:
# ratings_all[ratings_all['userID'] == 5702]

In [85]:
# is_target = (ratings_all['userID'] == 5702) & (ratings_all['itemID'] == 163)
# is_target

In [86]:
# ratings_all[is_target]

In [87]:
# target = ratings_all[is_target].iloc[0]
# target

In [88]:
##### CF korak po korak #####

In [89]:
#is_target = (ratings_all['userID'] == 1591) & (ratings_all['itemID'] == 11)
#is_target

In [90]:
#target = ratings_all[is_target].iloc[0]
#target

In [91]:
#train_set = sp.Dataset.load_from_df(ratings_all[~is_target][['userID', 'itemID', 'overall']], sp.Reader(rating_scale=(1, 5))).build_full_trainset()
#train_set

In [92]:
#algo = sp.KNNBasic(verbose=False)
#algo.fit(train_set)

In [93]:
#prediction = algo.predict(target['userID'], target['itemID'], verbose=False)
#prediction

In [94]:
#prediction.est

In [95]:
#prediction.est - target['overall']

In [96]:
#target['overall']

In [98]:
# Uzajamno filtriranje

def cf_predict(ratings, user_id, item_id):
    # pronalazi red u dataframe-u koji sadrzi prosledjeni user id i item id (to je red nazvan target)
    is_target = (ratings['userID'] == user_id) & (ratings['itemID'] == item_id)
    target = ratings[is_target].iloc[0]
    
    # kreiranje trening skupa podataka na osnovu svih redova osim target reda
    # izdvajaju se atributi 'userID', 'itemID', 'overall'
    train_set = sp.Dataset.load_from_df(
        ratings[~is_target][['userID', 'itemID', 'overall']], 
        sp.Reader(rating_scale=(1, 5))
    ).build_full_trainset()

    # kreiranje modela - koriscenje KNNBasic
    algo = sp.KNNBasic(verbose=False)
    # treniranje
    algo.fit(train_set)
    # predvidjanje ocene za proslednjeni user id i item id
    prediction = algo.predict(target['userID'], target['itemID'], verbose=False)
    
    # vracanje predvidjene ocene, razlike izmedju previdjene i stvarne ocene i stvarne ocene
    return prediction.est, prediction.est - target['overall'], target['overall']

In [99]:
cf_predict(ratings_all, 1591, 11)

(4.12550916496945, 1.1255091649694497, 3)

In [100]:
cf_predict(ratings_all, 5702, 163)

(4.125, -0.875, 5)

In [ ]:
#user_ratings = ratings_all[ratings_all['userID'] == 3958].join(items_all.set_index('id'), on='itemID', how='left', lsuffix='_left', rsuffix='_right')
#user_ratings

In [ ]:
#user_ratings['itemID_left'] == 106

In [ ]:
# ratings_all[ratings_all['userID'] == 5702]

In [ ]:
# user_ratings = ratings_all[ratings_all['userID'] == 5702].join(items_all.set_index('id'), on='itemID', how='left', lsuffix='_left', rsuffix='_right')
# is_target = (user_ratings['itemID_left'] == 163)
# is_target

In [101]:
# Filtriranje zasnovano na sadržaju

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier


def cb_predict(ratings, items, user_id, item_id):
    
    # kreiranje dataframe-a user_ratings sa informacijama o interakcijama
    # koje se odnose na korisnika sa datim id-em user_id i informacijama o povezanim proizvodima
    user_ratings = ratings[ratings['userID'] == user_id].join(items.set_index('id'), on='itemID',
                                                              how='left', lsuffix='_left', rsuffix='_right')
    # is_target kaze koji red iz dataframe-a user_ratings se odnosi na konkretan proizvod sa id-em item_id
    # 2260     True
    # 2563    False
    # 3241    False
    is_target = (user_ratings['itemID_left'] == item_id)
    
    # podela vrednosti atributa na trening i test (koriste se za predvidjanje, na kraju)
    # na primer ukoliko je pronadjeno 3 reda u kojima se javlja korisnik sa datim id-em,
    # dva ce ici u train_features i jedan (ako sadrzi dati item_id) ide u target_features
    features = pd.get_dummies(user_ratings.drop(columns=['overall']))
    train_features = features[~is_target]
    target_features = features[is_target]
    
    # enkodiranje vrednosti za atribut ocena (overall) koji se predvidja
    # npr. 5 --> 0
    encoder = LabelEncoder()
    train_labels = encoder.fit_transform(user_ratings[~is_target]['overall'])
    # target_label se ne enkodira, tu se samo smesta stvarna ocena koja treba biti predvidjena
    target_label = user_ratings[is_target]['overall'].iloc[0]
    
    # kreiranje modela, koriscenje KNeighborsClassifier
    clf = KNeighborsClassifier(n_neighbors=1)
    # treniranje modela na skupu za trening (prosledjuju se atributi i ciljna promenljiva - ocena,
    # koja je enkodirana)
    clf.fit(train_features, train_labels)

    # predvidjanje ocene koristeci target_features, a potom enkodiranje vrednosti nazad
    prediction = encoder.inverse_transform(clf.predict(target_features))[0]
    
    # vracanje predvidjene ocene, razlike izmedju predvidjene i prave ocene, prave ocene
    return prediction, prediction - target_label, target_label

In [102]:
cb_predict(ratings_all, items_all, 5702, 163)

(5, 0, 5)

In [46]:
def test_classifier(ratings, items, user_id, item_id):
    pred_cf, error_cf, truth = cf_predict(ratings, user_id, item_id)
    pred_cn, error_cn, truth = cb_predict(ratings, items, user_id, item_id)
    print("Rezultati za korisnika sa id-em {} za stavku sa id-em {}:".format(user_id, item_id))
    print("Pristup zasnovan na uzajamnom filtriranju: \t prediction: {:.5f} \t error: {:.5f}".format(pred_cf, error_cf))
    print("Pristup zasnovan na sadržaju: \t\t\t prediction: {:.5f} \t error: {:.5f}".format(pred_cn, error_cn))

In [47]:
test_classifier(ratings_all, items_all, 5702, 163)

Rezultati za korisnika sa id-em 5702 za stavku sa id-em 163:
Pristup zasnovan na uzajamnom filtriranju: 	 prediction: 4.12500 	 error: -0.87500
Pristup zasnovan na sadržaju: 			 prediction: 5.00000 	 error: 0.00000


### Implementacija hibridnih sistema preporuka

In [49]:
def predict_weighted(ratings, items, user_id, item_id):
    prediction_cf, _, true_rating = cf_predict(ratings, user_id, item_id)
    prediction_cb, _, true_rating = cb_predict(ratings, items, user_id, item_id)
    
    # Težine mogu biti odabrane drugačije,
    # prema tome koji model daje bolje preporuke.
    
    prediction = 0.5 * prediction_cf + 0.5 * prediction_cb
    error = prediction - true_rating
    
    return prediction, error, true_rating

In [58]:
pred_weighted, error_weighted, true_rating = predict_weighted(ratings_all, items_all, user_id=5702, item_id=163)
print("Hibrid sa težinama: \t predviđena ocena: {:.5f} \t greška: {:.5f}".format(pred_weighted, error_weighted))

Hibrid sa težinama: 	 predviđena ocena: 4.56250 	 greška: -0.43750


In [53]:
def predict_switching(ratings, items, user_id, item_id):
    
    # Odabir modela (sistema preporuka) je izvršen na osnovu
    # broja ocena koje su dostupne za datu stavku.
    
    num_ratings = len(ratings[ratings['itemID'] == item_id])
    if num_ratings > 3:
        print('Korišćenjem sistema zasnovanog na uzajamnom filtriranju')
        return predict_cf(ratings, user_id, item_id)
    else:
        print('Korišćenjem sistema zasnovanog na sadržaju')
        return predict_cn(ratings, items, user_id, item_id)

In [59]:
pred, error, true_rating = predict_switching(ratings_all, items_all, user_id=5702, item_id=163)
print("Hibrid sa smenjivanjem: \t predviđena ocena: {:.5f} \t greška: {:.5f}".format(pred, error))

Korišćenjem sistema zasnovanog na uzajamnom filtriranju
Hibrid sa smenjivanjem: 	 predviđena ocena: 4.12500 	 greška: -0.87500


In [103]:
# Provera da li postoje bar dva reda u dataframe-u sa korisnikom ciji je user_id = 3958
# ratings_all[ratings_all['userID'] == 3958]

In [104]:
# Provera da li postoje bar dva reda u dataframe-u sa stavkom ciji je item_id = 16
# ratings_all[ratings_all['itemID'] == 106]

In [ ]:
# items_all[items_all['itemID'] == 106]

In [60]:
pred, error, true_rating = predict_switching(ratings_all, items_all, user_id=3958, item_id=106)
print("Hibrid sa smenjivanjem: \t predviđena ocena: {:.5f} \t greška: {:.5f}".format(pred, error))

Korišćenjem sistema zasnovanog na sadržaju
Hibrid sa smenjivanjem: 	 predviđena ocena: 2.00000 	 greška: -3.00000


In [ ]:
# Hibridni sistem sa težinama daje dobre rezultate.

# Što se tiče hibridnog sistema sa smenjivanjem,
# izrazito bolje rezultate daje model zasnovan na uzajamnom filtriranju.